# Question-Answering Model

## Importing the libraries

In [1]:
from exchangelib import Credentials, Account
from datetime import datetime
import pytz
from transformers import pipeline
import torch
import numpy as np
import pandas as pd
import json
from pickle import dump

c:\Users\Ingrid Rodriguez\anaconda3\envs\outlook\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## EWS API to access Exchange Server Mailboxes

In [2]:
#Authenticate with Email and Password:
email = 'ingrid.test1@outlook.com'
password = 'Maslow94'

credentials = Credentials(email, password)
account = Account(email, credentials=credentials, autodiscover=True)

#Define Variables

In [12]:
# Define the queries
questions = [
    "What is the vendor's name",
    "What is the client's name",
    "What is the payment rate"
]
#Define Variables
email_answers_list = []
answers_dict = {}
tz = pytz.timezone('America/New_York')  # timezone


# Create a Pipeline for Question-Answering 

In [4]:
# # Create a pipeline for question answering
# qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad", tokenizer="bert-large-uncased")

qa_pipeline = pipeline(
    "question-answering",
    model="bert-large-uncased-whole-word-masking-finetuned-squad",
    tokenizer="bert-large-uncased",
    framework="pt"  # Explicitly specify PyTorch
)

c:\Users\Ingrid Rodriguez\anaconda3\envs\outlook\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ingrid Rodriguez\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when i

# Fetch Emails from a Date Range and feed the LLM

In [13]:
# Define the start and end date with timezone information
start_date = tz.localize(datetime(2023, 9, 21, 0, 0, 0))
end_date = tz.localize(datetime(2023, 10, 3, 23, 59, 59))
####################

# Get the current date in the Atlanta time zone
#current_date_atlanta = datetime.now(tz).date()
# Define the start and end date to cover today in the Atlanta time zone
#start_date = tz.localize(datetime(current_date_atlanta.year, current_date_atlanta.month, current_date_atlanta.day, 0, 0, 0))
#end_date = tz.localize(datetime(current_date_atlanta.year, current_date_atlanta.month, current_date_atlanta.day, 23, 59, 59))
###################

# Filter Emails based on the start and end dates
emails = account.inbox.filter(datetime_received__range=(start_date, end_date))
#emails.count()

5

In [14]:
#Iterate thorugh the emails' content to feed the LLM and get the answers to the defined queries in return

if not emails.count():
    print("No emails found in the QuerySet.")
else:
    # Iterate through the QuerySet and print the type of each item
    for email in emails:
        print(f"Type of item: {type(email)}")
        
        ############################################################
        # Get the Subject and Sender of the email
        print(f"Subject: {email.subject}")
        print(f"Sender: {email.sender.email_address}")
        
        # Get the plain text content of the email
        email_content=email.text_body

        if email_content:
            print(f"Plain Text Content:\n{email_content}")
            answers_dict = {}

            # Process each question and store the answers
            for question in questions:
                result = qa_pipeline(question=question, context=email_content)
                answer = result["answer"]
                answers_dict[question] = answer
            
            email_answers_list.append(answers_dict)

Type of item: <class 'exchangelib.items.message.Message'>
Subject: test
Sender: ingrid.test1@outlook.com
Plain Text Content:
The vendor is call vendor X while the client is called client C and my hourly rate is 10


Type of item: <class 'exchangelib.items.message.Message'>
Subject: Test3
Sender: ingrid.rodriguez.leon94@gmail.com
Plain Text Content:
Jsjsjs

Type of item: <class 'exchangelib.items.message.Message'>
Subject: Test 2
Sender: ingrid.rodriguez.leon94@gmail.com
Plain Text Content:
I live you Ingrid

Type of item: <class 'exchangelib.items.message.Message'>
Subject: Test1
Sender: ingrid.rodriguez.leon94@gmail.com
Plain Text Content:
The vendor is call vendor X while the client is called client C and my hourly rate is 10

Type of item: <class 'exchangelib.items.message.Message'>
Subject: Welcome to your new Outlook.com account
Sender: no-reply@microsoft.com
Plain Text Content:
[Microsoft]
Hi Ingrid,
Welcome to your Outlook! We're the email service designed to help you conquer yo

# FROM JSON TO DF

In [15]:
# Convert the list of dictionaries into a JSON structure
json_data = json.dumps(email_answers_list, indent=4)

# Convert the Json Structure into a DataFrame
df = pd.read_json(json_data)

# Rename the columns of the dataframe
df.columns = ["Vendor", "Client", "Payment Rate"]
df.index=df.index+1

C:\Users\Ingrid Rodriguez\AppData\Local\Temp\ipykernel_10684\3479137957.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data)


In [16]:
#Visualize the dataframe
df

,Vendor,Client,Payment Rate
1,call vendor X,client C,my hourly rate is 10
2,Jsjsjs,Jsjsjs,Jsjsjs
3,I live you Ingrid,I live you Ingrid,I live you Ingrid
4,call vendor X,client C,my hourly rate is 10
5,Microsoft,Ingrid,free


In [ ]:
# Save the DataFrame to a CSV file
#df.to_csv('data_vendors_clients.csv', index=False)

# Define the file name for download
#filename = 'data_vendors_clients.csv'

#...Download it into our system...#

# Outlook Function

In [24]:
def outlook_function (email, password, start_year, start_month, start_day, end_year, end_month, end_day):
    credentials = Credentials(email, password)
    account = Account(email, credentials=credentials, autodiscover=True)

    # Define the queries
    questions = [
    "What is the vendor's name",
    "What is the client's name",
    "What is the payment rate"
    ]
    #Define Variables
    email_answers_list = []
    answers_dict = {}
    tz = pytz.timezone('America/New_York')  # timezone

    #Define Pipeline
    qa_pipeline = pipeline(
        "question-answering",
        model="bert-large-uncased-whole-word-masking-finetuned-squad",
        tokenizer="bert-large-uncased",
        framework="pt"  # Explicitly specify PyTorch
    )

    # Define the start and end date with timezone information
    start_date = tz.localize(datetime(start_year, start_month, start_day, 0, 0, 0))
    end_date = tz.localize(datetime(end_year, end_month, end_day, 23, 59, 59))

    # Filter Emails based on the start and end dates
    emails = account.inbox.filter(datetime_received__range=(start_date, end_date))

    #Iterate thorugh the emails' content to feed the LLM and get the answers to the defined queries in return
    if not emails.count():
        print("No emails found.")
    else:
        # Iterate through the QuerySet and print the type of each item
        for email in emails:
            print(f"Type of item: {type(email)}")
            
            ############################################################
            # Get the Subject and Sender of the email
            #print(f"Subject: {email.subject}")
            #print(f"Sender: {email.sender.email_address}")
            
            # Get the plain text content of the email
            email_content=email.text_body

            if email_content:
                #print(f"Plain Text Content:\n{email_content}")
                answers_dict = {}

                # Process each question and store the answers
                for question in questions:
                    result = qa_pipeline(question=question, context=email_content)
                    answer = result["answer"]
                    answers_dict[question] = answer
                
                email_answers_list.append(answers_dict)
    
    # Convert the list of dictionaries into a JSON structure
    json_data = json.dumps(email_answers_list, indent=4)

    # Convert the Json Structure into a DataFrame
    df = pd.read_json(json_data)

    # Rename the columns of the dataframe
    df.columns = ["Vendor", "Client", "Payment Rate"]
    df.index=df.index+1

    return df

# Pickle the Function with the model

In [25]:
# save the model
dump(outlook_function, open('model.pkl', 'wb'))